# Imperative programming

In [1]:
def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

print(fancy_func(1, 2, 3, 4))

10


# Symbolic programming

computation is usually performed only once the process has been fully defined

1. Define the operations to be executed.

2. Compile the operations into an executable program.

3. Provide the required inputs and call the compiled program for execution.

In [2]:
def add_():
    return '''
def add(a, b):
    return a + b
'''

def fancy_func_():
    return '''
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
'''

def evoke_():
    return add_() + fancy_func_() + 'print(fancy_func(1, 2, 3, 4))'

prog = evoke_()
print(prog)
y = compile(prog, '', 'exec')
exec(y)


def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
print(fancy_func(1, 2, 3, 4))
10


# Hybrid Programming 混合

In [3]:
import torch
from torch import nn
from d2l import torch as d2l


# Factory for networks
def get_net():
    net = nn.Sequential(nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2))
    return net

x = torch.randn(size=(1, 512))
net = get_net()
net(x)

tensor([[-0.1437, -0.0678]], grad_fn=<AddmmBackward0>)

In [4]:
# By converting the model using torch.jit.script function, we are able to compile and optimize the computation in the MLP.
# The model’s computation result remains unchanged.

net = torch.jit.script(net)
net(x)

tensor([[-0.1437, -0.0678]], grad_fn=<AddmmBackward0>)

# Acceleration by Hybridization

In [5]:
#@save
class Benchmark:
    """For measuring running time."""
    def __init__(self, description='Done'):
        self.description = description

    def __enter__(self):
        self.timer = d2l.Timer()
        return self

    def __exit__(self, *args):
        print(f'{self.description}: {self.timer.stop():.4f} sec')

In [6]:
net = get_net()
with Benchmark('Without torchscript'):
    for i in range(1000): net(x)

net = torch.jit.script(net)
with Benchmark('With torchscript'):
    for i in range(1000): net(x)

Without torchscript: 0.1491 sec
With torchscript: 0.1276 sec


# Serialization

In [7]:
net.save('my_mlp')
!ls -lh my_mlp*

-rw-rw-r-- 1 tian tian 652K  2月 22 00:54 my_mlp


In [11]:
net2 = torch.load("my_mlp")

In [12]:
net2.eval()

RecursiveScriptModule(
  original_name=Sequential
  (0): RecursiveScriptModule(original_name=Linear)
  (1): RecursiveScriptModule(original_name=ReLU)
  (2): RecursiveScriptModule(original_name=Linear)
  (3): RecursiveScriptModule(original_name=ReLU)
  (4): RecursiveScriptModule(original_name=Linear)
)